# Scratchwork

In [1]:
from finite_algebras import *
from cayley_table import *
from permutations import *

In [2]:
import os
aa_path = os.path.join(os.getenv("PYPROJ"), "abstract_algebra")
alg_dir = os.path.join(aa_path, "Algebras")

In [3]:
ex = Examples(alg_dir)

                           Example Algebras
----------------------------------------------------------------------
  13 example algebras are available.
  Use "get_example(INDEX)" to retrieve a specific example,
  where INDEX is the first number on each line below:
----------------------------------------------------------------------
0: A4 -- Alternating group on 4 letters (AKA Tetrahedral group)
1: D3 -- https://en.wikipedia.org/wiki/Dihedral_group_of_order_6
2: D4 -- Dihedral group on four vertices
3: Pinter29 -- Non-abelian group, p.29, 'A Book of Abstract Algebra' by Charles C. Pinter
4: RPS -- Rock, Paper, Scissors Magma
5: S3 -- Symmetric group on 3 letters
6: S3X -- Another version of the symmetric group on 3 letters
7: V4 -- Klein-4 group
8: Z4 -- Cyclic group of order 4
9: F4 -- Field with 4 elements (from Wikipedia)
10: mag_id -- Magma with Identity
11: Example 1.4.1 -- See: Groupoids and Smarandache Groupoids by W. B. Vasantha Kandasamy
12: Ex6 -- Example 6: http://www-group

In [4]:
a4 = ex.get_example(0)
a4

Group(
'A4',
'Alternating group on 4 letters (AKA Tetrahedral group)',
['()', '(1 3 2)', '(1 2 3)', '(0 1)(2 3)', '(0 1 2)', '(0 1 3)', '(0 3 1)', '(0 3 2)', '(0 3)(1 2)', '(0 2 3)', '(0 2 1)', '(0 2)(1 3)'],
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [1, 2, 0, 6, 7, 8, 10, 11, 9, 5, 3, 4], [2, 0, 1, 10, 11, 9, 3, 4, 5, 8, 6, 7], [3, 4, 5, 0, 1, 2, 9, 10, 11, 6, 7, 8], [4, 5, 3, 9, 10, 11, 7, 8, 6, 2, 0, 1], [5, 3, 4, 7, 8, 6, 0, 1, 2, 11, 9, 10], [6, 7, 8, 1, 2, 0, 5, 3, 4, 10, 11, 9], [7, 8, 6, 5, 3, 4, 11, 9, 10, 0, 1, 2], [8, 6, 7, 11, 9, 10, 1, 2, 0, 4, 5, 3], [9, 10, 11, 4, 5, 3, 2, 0, 1, 7, 8, 6], [10, 11, 9, 2, 0, 1, 8, 6, 7, 3, 4, 5], [11, 9, 10, 8, 6, 7, 4, 5, 3, 1, 2, 0]]
)

In [5]:
f4 = ex.get_example(9)
f4

Field(
'F4',
'Field with 4 elements (from Wikipedia)',
['0', '1', 'a', '1+a'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]],
[[0, 0, 0, 0], [0, 1, 2, 3], [0, 2, 3, 1], [0, 3, 1, 2]]
)

In [6]:
ex6 = ex.get_example(12)
ex6

Ring(
'Ex6',
'Example 6: http://www-groups.mcs.st-andrews.ac.uk/~john/MT4517/Lectures/L3.html',
['0', 'a', 'b', 'c'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]],
[[0, 0, 0, 0], [0, 0, 1, 1], [0, 0, 2, 2], [0, 0, 3, 3]]
)

In [7]:
psr = generate_powerset_group(4)
psr.order

16

In [8]:
psr.about()


Group: PS4
Instance ID: 140398869687632
Description: Autogenerated Group on the powerset of 4 elements, with symmetric difference operator
Identity: {}
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {0}     {0}       2
      2     {1}     {1}       2
      3     {2}     {2}       2
      4     {3}     {3}       2
      5  {0, 1}  {0, 1}       2
      6  {0, 2}  {0, 2}       2
      7  {0, 3}  {0, 3}       2
      8  {1, 2}  {1, 2}       2
      9  {1, 3}  {1, 3}       2
     10  {2, 3}  {2, 3}       2
     11 {0, 1, 2} {0, 1, 2}       2
     12 {0, 1, 3} {0, 1, 3}       2
     13 {0, 2, 3} {0, 2, 3}       2
     14 {1, 2, 3} {1, 2, 3}       2
     15 {0, 1, 2, 3} {0, 1, 2, 3}       2
Group order is 16 > 12, so no further info calculated/printed.


In [9]:
# %time foo1 = psr.closed_subsets_of_elements(False)

In [10]:
# {len(x) for x in foo1}

In [11]:
# %time foo2 = psr.closed_subsets_of_elements(True)

In [12]:
# {len(x) for x in foo2}

In [13]:
# %time psr_subs = psr.unique_proper_subgroups()

In [14]:
# [psr_sub.order for psr_sub in psr_subs]